## Learning Objectives
In this tutorial, we will learn how to work with lidar derived raster product that represent topography and snow depth. The objectives include:
1. Understanding the basic concept of lidar remote sensing
2. Deriving snow products from lidar DEMs differencing
3. Analyzing lidar raster data
4. Identifying available lidar data for snow science

The prerequisite for this tutorial is basic experience with python and geospatial data. 

## Overview

We will start by exploring point cloud data and take a quick look at how rasters are created from them. Then we will derive vegetation and snow depth by differencing elevation models followed by analysing these lidar data including DEM. Finally, we will address common issue with working with lidar data from varying sources and talk about available snow focused lidar data.


## What is Lidar
Lidar stands for light detection and ranging also called 3D laser scanning. It is an active remote sensing technique that uses visible or near-infrared laser energy to measure elevation across wide areas. 

- Pulses of laser light are beamed to surface
- Return travel time determines distance to surface

<figure>
<img src = 'https://www.yellowscan-lidar.com/wp-content/uploads/2020/11/1280px-20200501_Time_of_flight.svg_.png' style="width:50%">
<figcaption align =  'center'><b> Time of flight measurement principle. Source: https://www.yellowscan-lidar.com/</b></figcaption>
</figure>


Lidar provides highly accuracte elevation data at high resolution
<figure>
<img src = 'https://i.imgur.com/QTc48KV.png' style="width:100%">
<figcaption align =  'center'><b>1 meter Lidar DEM vs 10 meter pixel USGS DEM. Source: Middlebury Remote Sensing</b></figcaption>
</figure>


There are different types depending on: platform used, physical process or scattering process. Based on the platform used, we find 3 types of LiDAR:
1. Ground-based LiDAR
2. Airborne LiDAR
3. Spaceborne LiDAR

For this tutorial, we will focus on Airborne LiDAR. Airborne LiDAR technology is used to measure topography using a laser beam directed towards the ground with GPS and IMU systems providing the location and orientation of the airborne platform. 

<figure>
<img src = 'https://gmv.cast.uark.edu/wp-content/uploads/2013/01/ALS_scematic.jpg' style="width:100%">
<figcaption align =  'center'><b>Source: https://gmv.cast.uark.edu/scanning-2/airborne-laser-scanning//</b></figcaption>
</figure>

## Discrete Vs Full Waveform Lidar

Lidar records two types of data- Range distance and Intensity. Range is a function of two-way travel time to ground and back. Intensity is the fraction of photons returned controlled by material properties.The distribution of returning energy creates a waveform with peaks at areas where more photons returns to the sensor. 

<figure>
<img src = 'https://pdal.io/en/stable/_images/return-pulse.png' style="width:100%">
<figcaption align =  'center'> Full waveform system records the distribution of the returned energy while discrete lidar records (red star) peaks in the waveform curve.   <b>Source: Neonscience.org</b></figcaption>
</figure>


These peaks or discrete points are called returns and often represent features on the ground like leaves, buildings, snow. The number and complexity of returns provide information about the surface material. 
<figure>
<img src = 'https://gisgeography.com/wp-content/uploads/2015/08/Number-Returns.png' style="width:100%">
<figcaption align =  'center'><b>Source: GISGeography</b></figcaption>
</figure>



## Georeferencing

The information collected by lidar scanners are referenced to the scanner coordinate system (SCS) and needs to be converted into real-world coordinate system. The process of rotating, translating and scaling lidar points is called georeferencing. The mathematics behind lidar georeferencing is contained in the equation described in [Glennie (2007)](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=7dc50f4fc427f2337e9b9b1d688248efeb041bb3). The equation coantains 14 unknowns that must be determined to georeference a single Lidar return. 

## The Point Clouds

Both discrete and full waveform lidar are provided as a collection of return lidar points known as point clouds. Points clouds are stored in a **.las** or **.laz** format. Each lidar point in a point cloud would have associated X, Y and Z values and other attributes depending on how the the data were collected and processed. The point clouds maybe also be classfied into ground/non-ground or multi-classes based on the different objects that each points reflected from. Lidar classification labels are numeric ranging from 0 to 20 or higher. Some common classes based on [ASPRS LAS specifications](https://www.asprs.org/wp-content/uploads/2010/12/LAS_Specification.pdf) is shown in the table below.

| label | Class |
| --- | --- |
| 0 | Not classified |
| 1 | Unassigned |
| 2 | Ground |
| 3 | Low Vegetation |
| 4 | Medium Vegetation |
| 5 | High Vegetation |
| 6 | Building |
| 7 | Noise |

Now, let's visualize and explore point clouds. In this activity, we will open a .las file, in the [plas.io free online lidar data viewer](http://plas.io/) and explore some attributes associated with a lidar data point cloud. 

1. Load the data: under file, select Autzen stadium. To navigate around, use left click and drag to tilt up and down, righ click and drag to move around. scroll bar to zoom in and out

2. Adjust Intensity 

3. Change the lidar point cloud color options to classification

4. Explore

## Point to Pixels

Points clouds give us a lot of information (x, y, z and intensity). However, these data might be too large, overwhelming and difficult for scientific application. Lidar data are often provided in raster data format which is easier to work with in many GIS tools. Raster data can be generated from point clouds using [PDAL](https://pdal.io/en/stable/) pipeline. The pipeline describes a series of processing stages to be performed in JSON format. In general, pipelines link together readers, filters and writers. A sample PDAL pideline for creating DEM raster using [Simple Morphological Filter (SMRF)](https://www.sciencedirect.com/science/article/pii/S0924271613000026) ground segmentation is shown below:

`
{
  "pipeline":[
  
    {    
      "type":"filters.reprojection",
      "out_srs":"EPSG:26911",
      "tag": "reprojectionUTM"
    },
    {
      "type":"filters.assign",
      "assignment":"Classification[:]=0",
      "tag": "reclassify"
    },
    {
      "type":"filters.elm",
      "tag": "noiseremoval"
    },
    {
      "type":"filters.outlier",
      "tag": "outlierremoval"
    },
    {
      "type":"filters.smrf",
      "ignore":"Classification[7:7]",
      "slope":0.2,
      "window":16,
      "threshold":0.45,
      "scalar":1.2,
      "tag": "groundsegmentation"
    },
    {
      "type":"filters.range",
      "limits":"Classification[2:2]",
      "tag":"classify"
    },
    {
       "type": "writers.las"
       "filename": "ground.laz",
       "inputs": [ "classify" ],
       "tag": "writerslas"       
        },
    {
        "type": "writers.gdal
        "filename": "DEM.tif",
        "gdalopts": "tiled=yes,
        "inputs": [ "writerslas" ],
        "nodata": -9999,
        "output_type": "idw",
        "resolution": 1
        }
  ]
}
`

